# Pre-processing Flood Inundation Map for Model Evaluation
### Created by: Sagy Cohen, University of Alabama; sagy.cohen@ua.edu

## Setting input data

In [6]:
import numpy as np
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True

#Workspace (preferably a Geodatabase what include all the input layers)
ws = arcpy.env.workspace = r'C:\ModelEvalWorkshop\ModelEvalWorkshop_Workspace_1.gdb' 

#DEM layer:
DEM = arcpy.Raster("DEM10m")

#Benchmark (remote sensing) FIM layer:
RSFIM = arcpy.Raster("RSFlood1014_orig")

#Permanent Water Bodies Layer:
PWB = arcpy.Raster("JRC_PWB")

#Final output name: 
finalOutputFIM = 'FinalFIM_RSFlood1014'

# Pre-processing
### Remove small clusters

In [7]:
# Convert benchmark FIM raster to [1, NoData]
tempFIM = arcpy.sa.Con(RSFIM==1,1,'#')
#initFIM.save('FIM_wNoData')

In [8]:
#Calculate clusters
RegionGroupRas = arcpy.sa.RegionGroup(in_raster=tempFIM, number_neighbors="EIGHT",zone_connectivity="WITHIN",add_link="ADD_LINK",excluded_value=None)
#RegionGroupRas.save('RegionGroup')

In [9]:
#Remove clusteres with a user-defined threshold
ClusterSizeTH = 200
classesStr = "1 {0} NODATA;{0} {1} 1".format(ClusterSizeTH,4057206)
tempFIM2 = arcpy.sa.Reclassify(in_raster=RegionGroupRas, reclass_field="Count", remap=classesStr, missing_values="DATA")
tempFIM2.save('FIM_'+str(ClusterSizeTH)+'removed')

initFIM = arcpy.sa.Con(tempFIM2==1,1,'#')
initFIM.save('FIM_wNoData')

## GapFill - a procedure to fill gaps in remote sensing FIM basaed on topography 

Step 1: Convert Remote Sensing to FIM [1, NoData] - NOTE: skip if ran in the pre-processing steps!!

Step 2: Resample DEM FIM resolution, extent and alignment; NOTE: there are two versions/choices of this step!!

Step 3: Get elevation to FIM from a DEM

Step 4: Run loop which

    Step 3.1: FocalStatistics [out_raster = arcpy.Raster("FocalSt_Elev_Iteration4")]
    Step 3.2: Filter high-and-dry pixels get new elevation [(arcpy.sa.Con(((out_raster - DEM) >=0),1,'#')) * (DEM + (out_raster-DEM))]

Step 5: Convert and save final output to FIM

### Step 1: Convert Remote Sensing to FIM [1, NoData] --- Skip if ran at the Pre-processing!!!

initFIM = arcpy.sa.Con(RSFIM==1,1,'#')
initFIM.save('FIM_wNoData')

#To Skip Step 1
initFIM = arcpy.Raster('FIM_wNoData')

### Step 2 v.1: Resample DEM FIM resolution, extent and alignment

cellSize='{} {}'.format(initFIM.meanCellHeight,initFIM.meanCellWidth)
with arcpy.EnvManager(outputCoordinateSystem=initFIM.spatialReference,snapRaster=initFIM):
    DEMr = arcpy.management.Resample(
        in_raster=DEM,
        out_raster='DEM_Resample',
        cell_size=cellSize,
        resampling_type="BILINEAR")

#To Skip step2
DEMr = arcpy.Raster('DEM_Resample')

### Step 2 v.2 - Resample FIM to DEM resolution and alignment

In [10]:
cellSize='{} {}'.format(DEM.meanCellHeight,DEM.meanCellWidth)
with arcpy.EnvManager(outputCoordinateSystem=DEM.spatialReference,snapRaster=DEM):
    initFIM = arcpy.management.Resample(
        in_raster=initFIM,
        out_raster='FIM_Resample',
        cell_size=cellSize,
        resampling_type="BILINEAR")
DEMr = DEM

### Step 3: Get elevation to FIM from a DEM

In [11]:
FIMelev = initFIM * DEMr
#FIMelev.save('FIMelev0')
initMean = arcpy.sa.IsNull(FIMelev).mean
print (initMean)
oldMean = initMean

0.9407374702558001


### Step 4: Run loop which
    Step 4.1: FocalStatistics [out_raster = arcpy.Raster("FocalSt_Elev_Iteration4")]
    Step 4.2: Filter high-and-dry pixels get new elevation [(arcpy.sa.Con(((out_raster - DEM) >=0),1,'#')) * (DEM + (out_raster-DEM))]

In [12]:
%%time
pdif = 100.0
iteration = 1
while pdif > 2:  
    print(iteration)
    out_raster = arcpy.sa.FocalStatistics(FIMelev, neighborhood="Rectangle 3 3 CELL", statistics_type="MAXIMUM", ignore_nodata="DATA", percentile_value=90)
    FIMelev = (arcpy.sa.Con(((out_raster -  DEMr) >= 0),1,'#')) * (DEMr + (out_raster-DEMr))
    isNull = arcpy.sa.IsNull(FIMelev)
    newMean = isNull.mean
    print (oldMean, newMean)
    FIMelev.save('FIMelev_3x3_GT0_10m'+str(iteration))
    pdif = 100*((initMean-newMean)-(initMean-oldMean))/(initMean-newMean)
    oldMean = newMean
    print(pdif)
    iteration+=1
print('out of the loop')

1
0.9407374702558001 0.9323269879040254
100.0
2
0.9323269879040254 0.9258822563454293
43.38363350053683
3
0.9258822563454293 0.9204178440412453
26.892287517916582
4
0.9204178440412453 0.915475659329764
19.563857579139594
5
0.915475659329764 0.910899080408487
15.337888353547207
6
0.910899080408487 0.9066543352171327
12.45409257845159
7
0.9066543352171327 0.9026325166071782
10.554582081481907
8
0.9026325166071782 0.8988241745984533
9.086238504982324
9
0.8988241745984533 0.8952485189867142
7.860492519573898
10
0.8952485189867142 0.8919201120364862
6.818080845905181
11
0.8919201120364862 0.8887926333531628
6.020769088534051
12
0.8887926333531628 0.8858651534304978
5.335076213357855
13
0.8858651534304978 0.8830888731905612
4.815867829003313
14
0.8830888731905612 0.8804521738052746
4.373702279875844
15
0.8804521738052746 0.8779707019631172
3.953480336260505
16
0.8779707019631172 0.8756210650902241
3.608363924449682
17
0.8756210650902241 0.873379560777315
3.3277521975722846
18
0.8733795607773

### Step 5: Convert and save final output to FIM

In [13]:
finalRas = arcpy.sa.Con(FIMelev, in_true_raster_or_constant=1, in_false_raster_or_constant=None, where_clause="VALUE > 0")
finalRas.save(finalOutputFIM)

# Post processing

### Classify Original RS map to [0, 1] from [1, NoData] 

In [14]:
RSreclass01 = arcpy.sa.Reclassify(initFIM,reclass_field="Value",remap="1 1;NODATA 0",missing_values="DATA")

### Calculate Evaluation layer: Reclass GapFill(ed) pixels to NoData

In [15]:
#finalRas = arcpy.Raster('FinalFIM_RGFlood1014_1pc_3x3_90pc_10m') ##bypass processing
finalEvalFIM1 = RSreclass01 + finalRas
finalEvalFIM2 = arcpy.sa.Reclassify(in_raster=finalEvalFIM1, reclass_field="Value", remap="1 NODATA;2 2;NODATA 0", missing_values="DATA")
finalEvalFIM3 = finalEvalFIM2 * 1
finalEvalFIM3.save(finalOutputFIM+'EvalLayer')

### Removing Permanent Water
#### Reclassify Permanent Water layer to [0,1]

In [16]:
ReclassPWB = arcpy.sa.Reclassify(PWB,reclass_field="Value",remap="1 1;NODATA 0",missing_values="DATA")
ReclassPWB.save('ReclassPWB')

#### Removing (make NoData) Permanent Water from the benchmark FIM

In [17]:
finalEvalFIM1 = ReclassPWB + finalEvalFIM3
finalEvalFIM2 = arcpy.sa.Reclassify(in_raster=finalEvalFIM1, reclass_field="Value", remap="0 0;1 NODATA;2 2;3 NODATA", missing_values="DATA")
finalEvalFIM3 = finalEvalFIM2 * 1
finalEvalFIM3.save(finalOutputFIM+'EvalLayer_PWBremoved')